# A simple RAG application using open-source models (Ollama/lmStudio) or OpenAI

### Note: Given that the LmStudio API calls are designed to work with the OpenAI SDK, we’re able to integrate the OpenAI SDK into LmStudio

In [1]:
# TYPE = "Ollama"
# MODEL = "gemma2:2b"


# TYPE = "OpenAI"
# MODEL = "gpt-3.5-turbo"
# OPENAI_KEY=""


TYPE= "LmStudio"
# MODEL = "gemma-3-4b-it"
MODEL = "Dorna2-Llama3.1-8B-Instruct"
EMBED_MODEL = "nomic-embed-text-v1.5"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from openai import OpenAI
from typing import List

if TYPE == "OpenAI":
    model = ChatOpenAI(
        openai_api_key=OPENAI_KEY,
        model=MODEL
    )
    embeddings = OpenAIEmbeddings()
elif TYPE == "Ollama":
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)
elif TYPE == "LmStudio":
    model = ChatOpenAI(
        openai_api_key="lm-studio",  # Can be any non-empty string for LM Studio
        model=MODEL,
        base_url="http://localhost:1234/v1"  # Default LM Studio API endpoint
    )

    # Embeding for LmStudio
    class LMSEmbeddingFunction:
        def __init__(self):
            self.client = OpenAI(
                base_url="http://localhost:1234/v1",
                api_key="lm-studio"
            )
        
        def embed_documents(self, texts: List[str], model=EMBED_MODEL) -> List[List[float]]:
            embeddings = self.client.embeddings.create(input=texts, model=model).data
            return [e.embedding for e in embeddings]
        
        def embed_query(self, text: str, model=EMBED_MODEL) -> List[float]:
            embedding = self.client.embeddings.create(input=[text], model=model).data[0].embedding
            return embedding

    # Create the embedding function
    embeddings = LMSEmbeddingFunction()
else:
    raise ValueError(f"Invalid Model Selection: {TYPE}")

# Only invoke if model is properly initialized
if model is not None:
    response = model.invoke("Tell me a joke")
    print(response)
else:
    print("Model not initialized")

content="Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 14, 'total_tokens': 31, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'dorna2-llama3.1-8b-instruct', 'system_fingerprint': 'dorna2-llama3.1-8b-instruct', 'id': 'chatcmpl-vy4uia3982d2k09x4xitm9', 'finish_reason': 'stop', 'logprobs': None} id='run-cbacbd6b-6248-4100-acec-2d65ac7a6549-0' usage_metadata={'input_tokens': 14, 'output_tokens': 17, 'total_tokens': 31, 'input_token_details': {}, 'output_token_details': {}}


In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

'Here\'s one:\n\nA man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?"\n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."'

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'Santiago'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data\\NikiCV.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Mozilla Firefox 136.0.2', 'creationdate': '2025-03-26T10:09:46+03:30', 'source': 'data\\NikiCV.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='M MO OH HA AM MM MA AD D  N NI IK KR RA AV VE ES SH H\nB B. .S Sc c. .\nC Co om mp pu ut te er r  S Sc ci ie en nc ce e\nU Un ni iv ve er rs si it ty y  o of f  S Se em mn na an n\n\uf073 S Se ep p  2 20 01 12 2  - -  N No ov v  2 20 01 17 7\n\uf3c5 S Se em mn na an n, ,  I Ir ra an n\nR Re el le ev va an nt t  C Co ou ur rs se es s\n- Data Structures\n- Algorithm Design\n- Database Management Systems\n- Computer Vision\n- Software Design Methodology\n• L Li in nu ux x  | |  D Do oc ck ke er r  | |  G Gi it t  , ,\n• N No od de e. .j js s  | |  J Ja av va aS Sc cr ri ip pt t  , ,\n• G Go oL La an ng g  , ,\n• P Py yt th ho on n  , ,\n• U Un ni it ty y3 3D D  ( (C C# #) )  , ,\n• N No oS SQ QL L  | |  S SQ QL L  , ,\n• C CI I/ /C CD D  | |

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\niki\Desktop\AI PROJECTS\mine_works_with_llms\.venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke("what is this pdf?")

[Document(metadata={'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Mozilla Firefox 136.0.2', 'creationdate': '2025-03-26T10:09:46+03:30', 'source': 'data\\NikiCV.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content='\uf111 D Da ad de eh h  G Go os st ta ar r \ue55f \ue55fS Se em mn na an n, ,  I Ir ra an n\nNode.js Linux Docker Git Redis MySQL MongoDB Angular2+ Ionic2+ WikiMedia\nS SO OF FT TW WA AR RE E  E EN NG GI IN NE EE ER R  ((B BA AC CK K- -E EN ND D  D DE EV VE EL LO OP PE ERR) ) Apr 2015 - Oct 2015\n• Back-end developer for several web and mobile\napplications.\n• Ported the "AMLAKNET" project to ASP MVC 3.\n• Designed and developed a cross-platform application called "TOUCHSI" for booking\ncars.\n• Wrote sample code and library for porting Java and Android push noti�cation.\nGitLab ASP MVC MSSQL Angular2+ Ionic2+ Windows Server JavaScript HTML CSS'),
 Document(metadata={'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Moz

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    "What is the purpose of the pdf?",
    # "how many pages is in this pdf?",
    # "How many coding assignments are there in the program?",
    # "Is there a program certificate upon completion?",
    # "What programming language will be used in the pdf?",
    "tell me a breif about it?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the pdf?
Answer: The PDF appears to be a resume or CV (Curriculum Vitae) created by Niki. The document provides an overview of Niki's education, work experience, skills, and personal projects. It highlights their expertise in software development, with specific mention of various programming languages, frameworks, and technologies they have worked with. The PDF also includes contact information and links to Niki's online presence.

The purpose of the PDF is likely to showcase Niki's qualifications, skills, and experience as a freelance developer and university graduate. It serves as a professional document for potential employers or clients to review Niki's background and consider them for job opportunities or projects.

Question: tell me a breif about it?
Answer: This is a resume or CV document created by Niki, a freelance developer. The document provides an overview of Niki's education, skills, and professional experience.

**Education:**
Niki graduat

In [12]:
chain.batch([{"question": q} for q in questions])

["The PDF appears to be a resume or CV (Curriculum Vitae) for a developer named Niki. The document showcases Niki's education, skills, work experience, and personal projects. It highlights their expertise in various programming languages and technologies such as Node.js, React Native, Angular, C#, JavaScript, Linux, Docker, MongoDB, Git, HTML, and CSS. The purpose of the PDF is likely to provide a comprehensive overview of Niki's qualifications and experience for potential employers or clients.",
 "This document appears to be a resume or CV for a freelance software developer named Niki. The CV highlights their education at the University of Semnan, where they earned a degree in Computer Science.\n\nThe majority of the document focuses on Niki's professional experience as a back-end developer. They have worked with various technologies such as Node.js, Angular, Ionic, React Native, Golang, C#, JavaScript, and MongoDB, among others. Their projects range from developing cross-platform app

In [13]:
for s in chain.stream({"question": "What is the purpose of the pdf?"}):
    print(s, end="", flush=True)

The PDF appears to be a resume or CV (Curriculum Vitae) document created by Niki. It outlines their educational background, work experience, skills, and personal projects. The document highlights their expertise as a freelance developer and showcases their diverse range of technical skills in various programming languages, frameworks, and technologies.

In [15]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.nikitv.ir")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents